In [29]:
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')

In [16]:
import GetOldTweets3 as got
import pandas as pd
import datetime as dt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import re
import time

stopWords = set(stopwords.words('english'))

In [21]:
username = '@JoeBiden'
# startDate = "2020-05-12"
# endDate = "2020-05-19"
start = dt.datetime.now() - dt.timedelta(days=1)
startDate = start.strftime('%Y-%m-%d')
endDate = dt.datetime.utcnow().strftime('%Y-%m-%d')

In [22]:
tweetCriteria = got.manager.TweetCriteria().setUsername(username).setSince(startDate).setUntil(endDate)
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

In [66]:
df = pd.DataFrame([tweet.__dict__ for tweet in tweets])
originalDF = df

df = df.drop(['author_id', 'date', 'geo', 'username'], axis=1)

In [67]:
df.sample(3)

,favorites,formatted_date,hashtags,id,mentions,permalink,replies,retweets,text,to,urls
0,56401,Tue Jun 02 23:12:00 +0000 2020,,1267957164607336448,,https://twitter.com/JoeBiden/status/1267957164...,3459,11326,The presidency is a big job. Nobody will get e...,None,
2,36335,Tue Jun 02 18:50:00 +0000 2020,,1267891230429626368,,https://twitter.com/JoeBiden/status/1267891230...,2070,8413,“I can’t breathe.” George Floyd’s last words. ...,None,
4,59468,Tue Jun 02 15:31:00 +0000 2020,,1267841150326636545,,https://twitter.com/JoeBiden/status/1267841150...,2684,12043,I’ve said from the outset of this election tha...,None,


## Finding number of important words (keywords)

In [68]:
# This function  returns count of nouns, verbs, aadjectives, adverbs in the given tweet
def getkeywordCount(tweet):
    # Remove non alphanumeric characters 
    tweet = re.sub(r'[^\w]', ' ', tweet)
    # make tokens
    wordTokens = word_tokenize(tweet)
    # remove stopwords &
    # perform lemmatisation 
    lemmatizer = WordNetLemmatizer() 
    filtered = [lemmatizer.lemmatize(w) for w in wordTokens if not w in stopWords]
    # tag parts of speech
    tagged = nltk.pos_tag(filtered)
    keywords = [w[0] for w in tagged if w[1][0] in ["N", "J", "V", "R"]]
    return len(keywords)

In [69]:
df['keywordCount'] = df.apply(lambda x : getkeywordCount(x["text"]), axis=1)

In [70]:
df.sample(3)

,favorites,formatted_date,hashtags,id,mentions,permalink,replies,retweets,text,to,urls,keywordCount
3,68216,Tue Jun 02 17:50:26 +0000 2020,,1267876242075746304,,https://twitter.com/JoeBiden/status/1267876242...,2009,13073,Julián — I made a promise to George's family t...,JulianCastro,https://twitter.com/JulianCastro/status/126785...,15
2,36335,Tue Jun 02 18:50:00 +0000 2020,,1267891230429626368,,https://twitter.com/JoeBiden/status/1267891230...,2070,8413,“I can’t breathe.” George Floyd’s last words. ...,None,,13
4,59468,Tue Jun 02 15:31:00 +0000 2020,,1267841150326636545,,https://twitter.com/JoeBiden/status/1267841150...,2684,12043,I’ve said from the outset of this election tha...,None,,18


## Normalise the columns

In [71]:
for feature in ["retweets", "favorites", "replies", 'keywordCount']:
    df[feature+"NormalisedCount"] = df[feature]/(df[feature].max())#-df[feature].min())

In [72]:
df.sample(3)

,favorites,formatted_date,hashtags,id,mentions,permalink,replies,retweets,text,to,urls,keywordCount,retweetsNormalisedCount,favoritesNormalisedCount,repliesNormalisedCount,keywordCountNormalisedCount
6,175765,Tue Jun 02 14:44:00 +0000 2020,,1267829322548563971,,https://twitter.com/JoeBiden/status/1267829322...,10619,30902,A country is crying out for leadership. Leader...,None,,20,0.509413,0.807146,0.741291,0.80
7,36541,Tue Jun 02 14:06:04 +0000 2020,,1267819776291692555,,https://twitter.com/JoeBiden/status/1267819776...,2877,9978,"Let us vow to make this, at last, an era of ac...",None,https://www.pscp.tv/w/caUCzDFvUEVMQkJiT2xyS2R8...,21,0.164485,0.167803,0.200838,0.84
8,30922,Tue Jun 02 02:55:00 +0000 2020,,1267650896562843648,,https://twitter.com/JoeBiden/status/1267650896...,1333,6227,"Pride is a global movement of love, self-expre...",None,https://medium.com/@JoeBiden/statement-by-vice...,24,0.102651,0.142000,0.093054,0.96


## Find out if the tweet is a retweet to some other persons tweet
#### We do this because retweeted tweets get a far greater reach and thus skews the scores

In [73]:
df["ifTo"] = df.apply(lambda x: 0.6 if x["to"] else 1,axis = 1)

In [74]:
df.sample(3)

,favorites,formatted_date,hashtags,id,mentions,permalink,replies,retweets,text,to,urls,keywordCount,retweetsNormalisedCount,favoritesNormalisedCount,repliesNormalisedCount,keywordCountNormalisedCount,ifTo
2,36335,Tue Jun 02 18:50:00 +0000 2020,,1267891230429626368,,https://twitter.com/JoeBiden/status/1267891230...,2070,8413,“I can’t breathe.” George Floyd’s last words. ...,None,,13,0.138686,0.166857,0.144503,0.52,1.0
8,30922,Tue Jun 02 02:55:00 +0000 2020,,1267650896562843648,,https://twitter.com/JoeBiden/status/1267650896...,1333,6227,"Pride is a global movement of love, self-expre...",None,https://medium.com/@JoeBiden/statement-by-vice...,24,0.102651,0.142000,0.093054,0.96,1.0
4,59468,Tue Jun 02 15:31:00 +0000 2020,,1267841150326636545,,https://twitter.com/JoeBiden/status/1267841150...,2684,12043,I’ve said from the outset of this election tha...,None,,18,0.198526,0.273088,0.187365,0.72,1.0


## Get time elapsed since every tweet.
### We hypothise that older tweets get more exposure

In [75]:
df["formatted_date"] = df.apply(lambda x: x["formatted_date"][:-10]+x["formatted_date"][-4:], axis=1)

df["timeElapsedHours"] = df.apply(lambda x: (pd.Timestamp(endDate) - pd.to_datetime(x["formatted_date"])).total_seconds()//3600, axis = 1 )

df["timeElapsedScore"] = 1+0.04*(7*12-df["timeElapsedHours"])/7/24

## Assign Score to tweet on basis of normalised count of retweets, favorites, replies, keyword count and time Elapsed

In [76]:
df["score"] = (1.5*df["retweetsNormalisedCount"] + 1.2*df["repliesNormalisedCount"] + 1*df["favoritesNormalisedCount"] + 2*df["keywordCount"])*df["ifTo"]*df["timeElapsedScore"]

In [77]:
df = df.drop(['id', 'keywordCount', 'retweetsNormalisedCount',
                  'favoritesNormalisedCount', 'repliesNormalisedCount',
                  'keywordCountNormalisedCount', 'ifTo',
                  'timeElapsedHours', 'timeElapsedScore'], axis=1)

In [78]:
sortedDF = df.sort_values(by=['score'], inplace=False,  ascending=False)

In [79]:
sortedDF.head(5)

,favorites,formatted_date,hashtags,mentions,permalink,replies,retweets,text,to,urls,score
0,56401,Tue Jun 02 23:12:00 2020,,,https://twitter.com/JoeBiden/status/1267957164...,3459,11326,The presidency is a big job. Nobody will get e...,None,,51.845400
8,30922,Tue Jun 02 02:55:00 2020,,,https://twitter.com/JoeBiden/status/1267650896...,1333,6227,"Pride is a global movement of love, self-expre...",None,https://medium.com/@JoeBiden/statement-by-vice...,49.133755
7,36541,Tue Jun 02 14:06:04 2020,,,https://twitter.com/JoeBiden/status/1267819776...,2877,9978,"Let us vow to make this, at last, an era of ac...",None,https://www.pscp.tv/w/caUCzDFvUEVMQkJiT2xyS2R8...,43.417242
6,175765,Tue Jun 02 14:44:00 2020,,,https://twitter.com/JoeBiden/status/1267829322...,10619,30902,A country is crying out for leadership. Leader...,None,,43.219044
5,137215,Tue Jun 02 14:58:00 2020,,,https://twitter.com/JoeBiden/status/1267832845...,5875,25300,The President held up a bible at St. John’s ch...,None,,40.457645


In [80]:
tweetsDF = sortedDF.head(5)

In [82]:
tweetsList = []
for i, row in df.iterrows():
    d = {}
#     d['username'] = row['username']
    d['date'] = row['formatted_date']
    d['replies'] = row['replies']
    d['retweets'] = row['retweets']
    d['favorites'] = row['favorites']
    d['text'] = row['text']
    d['link'] = row['permalink']
#     d['hashtag'] = row['hashtags']
#     d['mentions'] = row['mentions']
#     d['author_id'] = row['author_id']
    tweetsList.append(d)

In [83]:
tweetsList[0]

{'date': 'Tue Jun 02 23:12:00 2020',
 'replies': 3459,
 'retweets': 11326,
 'favorites': 56401,
 'text': 'The presidency is a big job. Nobody will get everything right. I won’t either. But I promise you this: I won’t traffic in fear and division or fan the flames of hate. I will seek to heal the racial wounds that have long plagued this country — not use them for political gain.',
 'link': 'https://twitter.com/JoeBiden/status/1267957164607336448'}